In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import math

In [2]:
img = cv2.imread('./paper4_checked.png')

## Get contours

In [3]:
def getContours(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #_, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 33)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CROSS, np.ones((3, 3)))
    contours, hierarchies = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    return contours, hierarchies

## Blocks
Blocks are considered as TestBlocks and Writing blocks

In [4]:
# Blocks
def countBlocks(contours, hierarchies, i):
    j = hierarchies[0][i][2]
    cnt = 0
    while j != -1:
        cnt += 1
        j = hierarchies[0][j][0]
    return cnt


def getRealParent(contours, hierarchies, numBlocks=12):
    areas = np.array([cv2.contourArea(contours[i]) for i in range(len(contours))], dtype='float32')
    indecies = np.argsort(areas)
    for i in indecies[::-1]:
        cnt = countBlocks(contours, hierarchies, i)
        if cnt == numBlocks: return i
    return None


def getBlocks(contours, hierarchies, parent):
    j = hierarchies[0][parent][2]
    lst = []
    while j != -1:
        lst.append(j)
        j = hierarchies[0][j][0]
    return lst


def getRectangles(contours):
    returns = []
    for i in range(len(contours)):
        contour = contours[i]
        rect = cv2.boundingRect(contour)
        x1, y1, x2, y2 = rect[0], rect[1], rect[2] + rect[0], rect[3] + rect[1]
        returns.append(((x1, y1), (x2, y2)))
    return returns


def getBlocksInOrder(contours, indecies):
    '''
    This method returns block indecies in order of their position.
    It will return the results in a way that number of questions are in order
    '''
    rects = getRectangles(contours)
    rects = sorted(zip(rects, range(len(rects))), key=lambda x: (x[0][0], x[0][1]))
    return [i[-1] for i in rects if i[-1] in indecies]


def approximatePolygon(contour):
    '''
    This method customly calculates the approximated polygon. Using this we can see
    whether the shape is a square or something else
    '''
    epsilon = 0.01 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, epsilon, closed=True)
    return approx

In [5]:
def getBlocksByImg(img, numBlocks=2, parent=None):
    contours, hierarchies = getContours(img) # get contours and hierarchies
    if parent is None:
        parent = getRealParent(contours, hierarchies, numBlocks=numBlocks) # get the parent to get children as blocks
    blocks = getBlocks(contours, hierarchies, parent) # find children as blocks
    res = getBlocksInOrder(contours, blocks) # sort for test blocks
    return contours, hierarchies, res

## Test boxes

In [10]:
AREA_LIMIT = 0.0045
POINT_LIMIT = 10
ASPECT_LIMIT = 1.1

def countSquares(contours, hierarchies, parent, imgShape):
    im_h, im_w = imgShape
    child = hierarchies[0][parent][2]
    cnt = 0
    while child != -1:
        area = cv2.contourArea(contours[child])
        rarea = area / (im_h * im_w)
        _, _, w, h = cv2.boundingRect(contours[child])
        aspectRatio = float(w) / h
        num_points = len(approximatePolygon(contours[child])) 
        if num_points <= POINT_LIMIT and num_points >= 4 and  rarea >= AREA_LIMIT and aspectRatio >= ASPECT_LIMIT:
            cnt += 1
        child = hierarchies[0][child][0]
    return cnt
    
    
def getTestBoxParent(contours, hierarchies, imgShape, squares=40):
    areas = np.array([cv2.contourArea(contours[i]) for i in range(len(contours))], dtype='float32')
    indecies = np.argsort(areas)
    for i in indecies[::-1]:
        cnt = countSquares(contours, hierarchies, i, imgShape)
        if  cnt == squares:
            return i
    print('NO PARENT FOUND')
    return None


def getTestSquares(contours, hierarchies, parent, imgShape):
    '''
    Return the indicies for the test squares inside each block
    '''
    im_h, im_w = imgShape
    child = hierarchies[0][parent][2]
    lst = []
    while child != -1:
        _, _, w, h = cv2.boundingRect(contours[child])
        area = cv2.contourArea(contours[child])
        rarea = area / (im_w * im_h)
        aspectRatio = float(w) / h
        num_points = len(approximatePolygon(contours[child])) 
        if num_points <= POINT_LIMIT and num_points >= 4 and  rarea >= AREA_LIMIT and aspectRatio >= ASPECT_LIMIT:
            lst.append(child)
        child = hierarchies[0][child][0]
    return lst


def sortTestSquares(contours, square_indicies):
    rects = getRectangles(contours)
    rects = list(zip(range(len(rects)), rects))
    rects = sorted(rects, key=lambda x: (x[1][0][1] // 10, x[1][0][0] // 10))
    return [x[0] for x in rects if x[0] in square_indicies]


def checkSquare(square):
    if np.mean(square / 255.0) > 0.5:
        return 1
    return 0

In [7]:
def getTestSquaresByImg(img, numSquares=40, parent=None):
    contours, hierarchies = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    if parent is None:
        parent = getTestBoxParent(contours, hierarchies, img.shape, squares=numSquares)
    squares = getTestSquares(contours, hierarchies, parent, img.shape)
    squares = sortTestSquares(contours, squares)
    return contours, hierarchies, squares

In [8]:
def test_results(img):
    contours, hierarchies, blocks = getBlocksByImg(img)
    rects = getRectangles(contours)
    i = 0 # Test part

    question = 1
    squares_checked = 0
    buffer = [0, 0, 0, 0]
    questions = {}
    
    # Get test Blocks
    tblocks = getBlocks(contours, hierarchies, blocks[i])
    tblocks = getBlocksInOrder(contours, tblocks)
    trects = getRectangles(contours)
    for j in tblocks:
        tpoints = trects[j]
        test_block = img[tpoints[0][1]:tpoints[1][1], tpoints[0][0]:tpoints[1][0], :].copy()
        test_block = cv2.cvtColor(test_block, cv2.COLOR_BGR2GRAY)
        test_block = cv2.adaptiveThreshold(test_block, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 33, 33)
        test_block = cv2.morphologyEx(test_block, cv2.MORPH_CROSS, (11, 11))
        test_block = cv2.morphologyEx(test_block, cv2.MORPH_CLOSE, (7, 7))
        # Get squares
        scontours, shierarchies, sblocks = getTestSquaresByImg(test_block)
        srects = getRectangles(scontours)
        
        for k in sblocks:
            spoints = srects[k]
            square = test_block[spoints[0][1]:spoints[1][1], spoints[0][0]:spoints[1][0]].copy()
            result = checkSquare(square)
            buffer[squares_checked] = result
            squares_checked += 1

            if squares_checked == 4:
                squares_checked = 0
                if sum(buffer) > 1: questions[question] = -1
                elif sum(buffer) == 0: questions[question] = 0
                else: questions[question] = np.argmax(buffer) + 1
                question += 1
                buffer = [0, 0, 0, 0]
    return questions

In [13]:
contours, hierarchies, blocks = getBlocksByImg(img)
rects = getRectangles(contours)
i = 1 # Writing part